In [1]:
import matplotlib.pyplot as plt
from graphics import *
from operator import *
from random import *
import numpy as np
import timeit
from math import *

a = 3
s = 3
r = 3
rr = 20
ro = 70
ra = 300
noise = 0.01
prop = 0.4
weight = 3
bias = np.array([0.0,-1.0])
dev_bias = 3
dTheta = 120
sight_theta = 180


In [39]:
def initialize_agents(speed, N, width, height):
    """
    Initializes our agent set with randomly directed speeds, draws the window and the agents
    """
    seed()

    agents = [np.array([0.0, 0.0]) for i in range(N)]
    #agents = [Point(uniform(0, width), uniform(0, height)) for i in range(N)]
    
    speeds = N*[0,0]
    #speeds = [np.array([0.0, 0.0]) for i in range(N)]
    
    for i in range(N):
        theta = uniform(0, 2 * np.pi)
        #speeds[i][0] = speed * np.cos(theta)
        #speeds[i][1] = speed * np.sin(theta)
        speeds[i]= speed * np.array([np.cos(theta),np.sin(theta)])
        
        theta = uniform(0, 2 * np.pi)
        radius = uniform(0,height/2)
        agents[i] = Point(width/2 + radius*np.cos(theta), height/2 + radius*np.sin(theta))

    return agents, speeds


def initialize_window(agents, width, height):
    win = GraphWin("Swarm", width, height) # size of box
    for agent in agents:
        agent.draw(win)
    win.getMouse()
    return win


def couple_speeds(agents, speeds, a, s, N):
    """
    Simplest model: for each agent, it will give its nearest neighbour a fraction of its speed and re-normalize it
    
    """
    nearest_neighbours = [nearest_neighbour(agent, agents, N) for agent in agents]
    for i in range(N):
        weightedSpeed = a * speeds[nearest_neighbours[i]]
        speeds[i] = speeds[i] + weightedSpeed
        speeds[i] = s * normalized(speeds[i])


def angle(vec1, vec2):  # (Not so) stupid way of doing an angle
    return np.arccos(np.dot(normalized(vec1),normalized(vec2))) # radians
            
def get_distances(agent, agents, N):
    """
    Given one angent and the set of all agents, 
    computes the distances from the first to all of the others
    """
    dists = N * [0.0]
    for i in range(N):
        a, dists[i] = relative_pos(agent, agents[i])
        if dists[i] == 0:
            dists[i] = 0.1
    
    return dists

def relative_pos(agent1, agent2):
    dx = agent2.getX() - agent1.getX()
    dy = agent2.getY() - agent1.getY()

    return np.array([dx, dy]), np.linalg.norm([dx, dy])

def nearest_neighbour(agent, agents, N):
    """
    Returns the index for the agent with smallest Eucledian distance to agent in question
    """
    distances = get_distances(agent, agents, N)
    j = next(i for i in range(N) if agents[i] is agent)
    distances[j] = distances[j-1] + 1

    return distances.index(min(distances))


def softened_angle(speed, newspeed, s, maxTheta):
    theta = angle(speed, newspeed)
    if maxTheta**2 > theta**2:
        return s*newspeed
    else:
        return np.dot(rot_matrix(maxTheta), speed)    

def in_sight_range(rel_pos, speed1, angle_range):
    return 360*angle(speed1, rel_pos)/np.pi < angle_range
    

def noisy_vector(noise):
    return noise * np.array([2 * random() - 1, 2 * random() - 1])
    
def biaser(speeds, N, s, i, prop, bias, dev_bias, weight):
    #bias = np.array([0.0,1.0])
    #Ns has to be integer 
    Ns = int(N * prop)
    gbias = np.random.normal(bias, dev_bias, )
    for i in range(Ns):
        tot_dir = normalized(normalized(speeds[i]) + weight * gbias)
        
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir
    #bias = np.dot(tot_dir,np.array([[np.cos(rot_bias*i), 0],[0, np.sin(rot_bias*i)]]))
    #return bias

def quality (agents, speeds, N, bias, window, old_cm):
#ACCURACY (DIRECTIONS DISTRIBUTION)
    dev = 0.0
    for i in range(N):
        dev += angle(bias, speeds[i])
    dev_avg = dev/(N*2*(np.pi))
    #print (dev_avg)
    
#ELONGATION: SHAPE OF SWARM
    #can be smarter if we make agents become poses before, more globally in  the code
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    #Center of Mass
    cm = np.mean(poses,axis=0)
    #Standard Deviation
    std = np.std(poses,axis=0)
    #print (cm, std)

    #Elongation >1 means 
    elong = std[1] / std[0]
    #Drawing it
    #old_cm = Point(cm[0],cm[1])
    #old_cm.draw(window)
    #old_cm.setFill("red")

#GROUP DIRECTION
    #Vector
    group_dir = np.array([cm[0] - old_cm[0], cm[1] - old_cm[1]])                
    #print (group_dir)
    # Norm
    group_dir = np.linalg.norm(group_dir)
    #print (group_dir)
    return cm
    


def rigid_boundary(x_bound, y_bound, agents, speeds, N):
    for i in range(N):
        [dx, dy] = [0, 0]
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            speeds[i][0] = -speeds[i][0]
            dx = x_bound - x         
        
        elif x < 0:
            speeds[i][0] = -speeds[i][0]
            dx = -x
        
        if y > y_bound:
            speeds[i][1] = -speeds[i][1]
            dy = y_bound - y
        
        elif y < 0:
            speeds[i][1] = -speeds[i][1]
            dy = -y

        agents[i].move(dx, dy)
        
        
def periodic_boundary(x_bound, y_bound, agents, speeds, N):  #Changed from rigid boundaries do periodic boundary condition
    [dx, dy] = [0, 0]
    for i in range(N):
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            dx = -x_bound         
        
        elif x < 0:
            dx = x_bound
        
        if y > y_bound:
            dy = -y_bound
        
        elif y < 0:
            dy = y_bound

        agents[i].move(dx, dy)

def next_step(agents, speeds, dt, N):
    dxvec = [dt * speeds[i][0] for i in range(N)]
    dyvec = [dt * speeds[i][1] for i in range(N)]
    for i in range(N):
        agents[i].move(dxvec[i], dyvec[i])
        

def normalized(vector):
    if vector[0] == vector[1] == 0:
        return vector
    return vector / np.linalg.norm(vector)


def rot_matrix(theta, unit = "None"):
    if unit in ["None", "deg"]:
        c, s = np.cos(pi * theta / 180), np.sin(pi * theta / 180)
    elif unit == "rad":
        c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s],[s, c]])

def warn_me_args(N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dTheta):
    if dt*s < rr:
        print("Warning - step length bigger the repulsion radius.")
        

        
def virtualizer (current, agents, h, w, N):
    lower_limit = current.getY() - h / 2
    upper_limit = current.getY() + h / 2
    left_limit = current.getX() - w / 2
    right_limit = current.getX() + w / 2
    
    vagents=[np.array([agent.getX(),agent.getY()]) for agent in agents]
    virtuals = N * [0.0, 0.0]
    vvirtuals = N * [Point(0.0, 0.0)]

    for j in range(N):
        #make more compact!!!
        virtuals[j]=vagents[j]
        
        #newcopy = agents[i].clone()
        candidates = [vagents[j],
                      vagents[j]+[w,0],vagents[j]+[-w,0],vagents[j]+[w,h],vagents[j]+[w,-h],
                      vagents[j]+[-w,h],vagents[j]+[-w,-h],vagents[j]+[0,h],vagents[j]+[0,-h]]
        #print candidates
        
        #virtuals[j] = next((cand for cand in candidates if lower_limit < cand[1] < upper_limit and left_limit < cand[0] < right_limit),False
        for i in range(9):
            if lower_limit < candidates[i][1] < upper_limit and left_limit < candidates[i][0] < right_limit:
                virtuals[j]=candidates[i]
            
        vvirtuals[j] = Point(virtuals[j][0],virtuals[j][1])
        
    return vvirtuals


def get_cm_std(agents, N):
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    return np.mean(poses,axis = 0), np.std(poses,axis = 0)

def mill_observables (N, agents, speeds):
    cm, std = get_cm_std(agents, N)
    mean_R = np.linalg.norm(std)
    point_cm = Point(cm[0],cm[1])
    norm_R = N*[0.0]
    vector_R = N*[0,0]
    versor_R = N*[0,0]
    versor_T = N*[0,0]
    proj_radial = N*[0]
    radial = N*[0,0]
    proj_tangencial= N*[0]
#    norm_tangencial= N*[0]
    tangencial= N*[0,0]
    angles = N*[0.0]
    mean_angle = 0.0
    mean_speed = 0.0
    mean_tangencial = 0.0
    mean_radial = 0.0
    mean_square = 0.0
    angular= N*[0.0]
    
    for i in range(N):
        vector_R[i],norm_R[i] = relative_pos(point_cm, agents[i])
        
        versor_R[i] = normalized(vector_R[i])
        proj_radial[i] = np.dot(speeds[i], versor_R[i])
        radial[i] = proj_radial[i]*versor_R[i]
        
        angular[i] = (1/(norm_R[i])**2)*np.cross([vector_R[i][0],vector_R[i][1],0],[speeds[i][0],speeds[i][1],0])[2]
        #norm_tangencial[i]= norm_R[i]*angular[i]
        
        versor_T[i] = np.dot(versor_R[i],rot_matrix(90,'deg'))
        proj_tangencial[i] = np.dot(speeds[i], versor_T[i])
        tangencial[i] = proj_tangencial[i]*versor_T[i]
        
        
        angles[i] = (np.pi - angle(vector_R[i],speeds[i]))*360/(2*np.pi)
        mean_angle += angles[i]
        mean_tangencial += abs(proj_tangencial[i])
        mean_radial += abs(proj_radial[i])
        mean_speed += np.linalg.norm(speeds[i])
        mean_square += (np.linalg.norm(speeds[i]))**2
    
        #print proj_radial[i], proj_tangencial[i]
        #print angles[i], angular[i]
    mean_angle = mean_angle / N
    mean_speed = mean_speed / N
    mean_radial = mean_radial / N
    mean_tangencial = mean_tangencial / N
    mean_square = mean_square / N
    print mean_speed, mean_tangencial, mean_radial
    #print np.linalg.norm(speeds[0]), square # square is converging to alpha / beta !!! amazing!
    #print norm_tangencial[0] # norm_tangencial should be the same as proj_tangencial 
    
    min_R = min(norm_R)
    max_R = max(norm_R)
    
    #return mean_R, min_R, max_R, 
    #return angles



In [40]:

##MILL MODEL
def mill (dt, agents, speeds, N, width, height, cr, ca, lr, la, alpha, beta, mass):
    clr = cr / lr
    cla = ca / la
    for i in range(N):
        u_dir = np.array([0.0, 0.0])
        propulsion = np.array([0.0, 0.0])
        friction = np.array([0.0, 0.0])
        grad_U = np.array([0.0, 0.0])
        
        #virtuals = virtualizer(agents[i], agents, height, width, N)
        
        for j in range(N):   #Duality interactions, by the Morse potential  
            if i == j:
                #Eliminate the i-i interaction
                continue

            rel_pos, distance = relative_pos(agents[i], agents[j])
            u_dir = normalized(rel_pos)
            grad_U = grad_U + u_dir * (clr*np.exp(-distance / lr) - cla *np.exp(- distance / la))
            
        
        propulsion = alpha * speeds[i] # self-propulsion propto alpha
        norm = (np.linalg.norm(speeds[i]))
        friction =  beta *((norm)**2) * speeds[i] #friction force propto beta
     
        d_speed = (propulsion - friction - grad_U)/mass
        speeds[i]= speeds[i]+dt*d_speed
        #print u_dir
    
   # print speeds[N-1], propulsion, friction, grad_U
    return

def run(N_steps, N, s, dt, width, height):#N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dev_bias, dTheta, sight_theta):
    """
    Simulates motion of swarm. Recieves following parameters:

    N_steps  - number of steps to perform
    a -  coupling between neighbouring points
    dt - time step to be used 
    N - number of points to be used
    width & heigth of window
    s - module of speed throughout agents

    """
    
    agents, speeds = initialize_agents(s, N, width, height)
    window = initialize_window(agents, width, height)
    old_cm=np.array([0.0,0.0])
    bias = np.array([0.0,-1.0])
    for i in range(N_steps):
        
        #print 'step', i
        
        next_step(agents, speeds, dt, N)
        start = timeit.default_timer()
        ## MODEL
        #couple_speeds(agents, speeds, a, s, N)
        #vicsek(agents, speeds, N, s, 0.1, r = 500)
        #couzin(agents, speeds, N, width, height, s, noise, dTheta, rr, ro, ra, sight_theta, 1, 200, 1.5, 1) #(...)last 4 parameters: model2, roa, orient, atract
        
        mill (dt, agents, speeds, N, width, height, cr=120, ca=250, lr=120, la=250.0, alpha=1.0, beta=0.01, mass=1) 
        # Region II, really nice ring
        #mill (dt, agents, speeds, N, width, height, cr=110, ca=100, lr=20, la=100.0, alpha=4.0, beta=0.06, mass=3) 
        # Region VII,(l<1/sqrt(C)) alpha ~ beta, best vortex in this region, decreasing in size with increasing N
        
        mill_observables (N, agents, speeds)
        print i
        #print angles
        
        
        #interaction(agents, speeds)
        #biaser(speeds, N, s, i, prop, bias, dev_bias, weight)
        
        ## BOUNDARY CONDITIONS
        rigid_boundary(width, height, agents, speeds, N)
        #periodic_boundary(width, height, agents, speeds, N)

        #time.sleep(0.02)
        stop = timeit.default_timer()
        #print stop - start
    window.close()
    return

In [41]:

run(100000, 20, 5, 0.3, 900, 700)

6.24908613524 4.48792136487 3.69686580735
0
7.56182873066 5.35244254738 4.39899797033
1
8.70603110696 6.1028988356 5.10548802832
2
9.50597404283 6.63204793793 5.66221794274
3
9.98374694473 6.9372598142 6.05512086577
4
10.2577256606 7.0848365537 6.30166481202
5
10.4196376739 7.13998358042 6.45996709139
6
10.5219012098 7.14435097309 6.5833402699
7
10.591842483 7.10717753308 6.63353651589
8
10.6432286485 6.99427413914 6.86559629848
9
10.6831414391 6.78520099637 7.26534508261
10
10.7153302914 6.54100658446 7.56313157363
11
10.7417860628 6.31162804383 7.77831434044
12
10.7635690296 6.10483996438 7.98547092367
13
10.7812722187 5.91660450276 8.17574370647
14
10.7952682642 5.74329116811 8.33664202737
15
10.8058302771 5.58316221439 8.47144314894
16
10.8131880696 5.44218963427 8.58170459152
17
10.8175537865 5.32151576371 8.66791197475
18
10.8191283617 5.21647420149 8.72934520784
19
10.8180943517 5.13049229477 8.76306326792
20
10.8146077037 5.0688047074 8.76096141776
21
10.8087959874 5.0512064597

10.0184495088 8.60255934354 4.37241472561
186
10.0049564915 8.59575130075 4.27495661549
187
9.99037579792 8.59381815108 4.11925153597
188
9.97475596804 8.59446311135 3.94001968495
189
9.9582227769 8.59355967507 3.83444608301
190
9.94107762479 8.58742801468 3.6960799899
191
9.92375678118 8.57438967339 3.74379507141
192
9.90671395042 8.55508808342 3.83039276046
193
9.89031285858 8.53167698751 3.91258844455
194
9.87487204791 8.50673240175 3.97384104675
195
9.86069616329 8.48245608388 3.9925516492
196
9.84813705477 8.46050921002 3.97336474578
197
9.83744167971 8.44208279212 4.02004195757
198
9.82869669672 8.42808517902 4.03121701004
199
9.82188370621 8.41930022162 4.04819614755
200
9.816915815 8.41638806018 4.03426057173
201
9.81366754212 8.4198334803 3.99609667943
202
9.81201560804 8.42994346176 4.00738367049
203
9.81189046117 8.44690502349 3.99961063341
204
9.81324730253 8.47080932675 3.9640751207
205
9.81603471816 8.50173618555 3.90278312206
206
9.8201339106 8.53983403082 3.8296399355
2

10.0287859606 9.89809147405 1.36759674609
380
10.0332509559 9.90288616943 1.39665739459
381
10.0369612548 9.90637680267 1.41203073819
382
10.039961267 9.90852312145 1.41343531039
383
10.042305595 9.9093023656 1.4010679106
384
10.0439114452 9.90857817599 1.3991471617
385
10.0446926439 9.90624812887 1.39552926325
386
10.0446737025 9.90237971452 1.39900343231
387
10.0438753407 9.89709784675 1.40177438238
388
10.0422676157 9.89053645398 1.40291209087
389
10.0398544339 9.88292213097 1.39556554665
390
10.0367382874 9.87462865892 1.37667107168
391
10.0330918 9.86613453352 1.35679776386
392
10.0290616223 9.85790500461 1.34129990538
393
10.0249447829 9.85055233896 1.35982867319
394
10.0211527177 9.84478260874 1.36606437079
395
10.0180065576 9.84116990669 1.3581769286
396
10.0156867193 9.84008199916 1.34956181048
397
10.0142152914 9.84165186376 1.33794080668
398
10.0134925642 9.84579687321 1.32081089363
399
10.0132722807 9.85217803835 1.29031115731
400
10.0132776432 9.86031071159 1.24488104828
4

9.98284959124 9.944315902 0.705253975422
558
9.98362744332 9.94472370409 0.715171544219
559
9.98458488391 9.94537755672 0.730263062063
560
9.98580905477 9.94649739336 0.73856769705
561
9.98730217368 9.94820549573 0.756339033152
562
9.98894206996 9.95046899154 0.764489653082
563
9.99046873148 9.95307576371 0.765034868416
564
9.99164043582 9.95577935793 0.757280668492
565
9.99225205797 9.95831329985 0.739336497719
566
9.9920851771 9.96034249329 0.710456529617
567
9.99106631143 9.96163057783 0.671248161104
568
9.98937686825 9.96216400546 0.622554792733
569
9.98739197112 9.96210369609 0.573503249881
570
9.98547560988 9.96159819406 0.547128381389
571
9.98389262408 9.96071697096 0.551031350841
572
9.98285367708 9.959512812 0.559615708464
573
9.98244422066 9.95796253109 0.571301003365
574
9.98277181831 9.95612520782 0.57985296445
575
9.98381097937 9.95399367385 0.594059296711
576
9.98546375976 9.95155531483 0.610566397003
577
9.98763485254 9.94885889886 0.654207871705
578
9.99027887463 9.9460

9.9994476036 9.96642881764 0.655812730862
738
10.001983272 9.97171200779 0.608737155162
739
10.0042706651 9.97610543091 0.584194509283
740
10.0062266276 9.97951327437 0.579511411914
741
10.0077386428 9.9818672651 0.575324569489
742
10.0086531825 9.98310006412 0.5793137549
743
10.0087820311 9.98313567348 0.590034718847
744
10.0079619437 9.98193606647 0.601487085626
745
10.0061492034 9.97958396672 0.605321160712
746
10.003367726 9.9762232882 0.601809359818
747
9.99987919161 9.97222351503 0.591957693451
748
9.99620921462 9.96820632232 0.580595807306
749
9.99279779902 9.96469193996 0.584196177996
750
9.98986394443 9.96195780325 0.597332645351
751
9.98747621055 9.96010408862 0.6038050334
752
9.98586731957 9.95936723154 0.60332437834
753
9.9851551287 9.9598422518 0.595562899517
754
9.98540760158 9.96155118988 0.580680744193
755
9.98671104833 9.96451437915 0.558877177016
756
9.98904504704 9.96862646819 0.53039338974
757
9.99231165701 9.97368714783 0.508012866048
758
9.99629585704 9.9793653943

KeyboardInterrupt: 

KeyboardInterrupt: 